In [2]:
import os
from PIL import Image, ImageOps, ImageEnhance
import random
import numpy as np
from rembg import remove
import io

In [3]:
base_dir = "../datasets"
train_dir = os.path.join(base_dir, "train")

In [8]:
def rotate_image(image_path, save_dir, angles=[-15, -10, -5, 5, 10, 15]):
    image = Image.open(image_path)
    for angle in angles:
        rotated_image = image.rotate(angle, expand=True)
        # Save the rotated image with a new name
        base_name = os.path.basename(image_path)
        name, ext = os.path.splitext(base_name)
        new_name = f"{name}_rotated_{angle}{ext}"
        rotated_image.save(os.path.join(save_dir, new_name))

def flip_image(image_path, save_dir):
    image = Image.open(image_path)
    flipped = ImageOps.mirror(image)  # Horizontal flip
    # Save
    base_name = os.path.basename(image_path)
    name, ext = os.path.splitext(base_name)
    new_name = f"{name}_flip{ext}"
    flipped.save(os.path.join(save_dir, new_name))

# Function to scale (zoom-in) and save images
def scale_image(image_path, save_dir, scale_factor=0.9):
    image = Image.open(image_path)
    width, height = image.size
    
    # Calculate cropping box
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    
    left = (width - new_width) // 2
    top = (height - new_height) // 2
    right = left + new_width
    bottom = top + new_height
    
    cropped = image.crop((left, top, right, bottom))
    scaled = cropped.resize((width, height), Image.Resampling.LANCZOS)
    
    # Save
    base_name = os.path.basename(image_path)
    name, ext = os.path.splitext(base_name)
    new_name = f"{name}_scale{ext}"
    scaled.save(os.path.join(save_dir, new_name))

def adjust_brightness(image_path, save_dir, factor_range=(0.7, 1.3)):
    image = Image.open(image_path)
    
    enhancer = ImageEnhance.Brightness(image)
    factor = random.uniform(*factor_range)  # Random brightness factor
    brightened = enhancer.enhance(factor)
    
    # Save
    base_name = os.path.basename(image_path)
    name, ext = os.path.splitext(base_name)
    factor_tag = f"{factor:.2f}".replace('.', '')
    new_name = f"{name}_bright{factor_tag}{ext}"
    brightened.save(os.path.join(save_dir, new_name))

def adjust_contrast(image_path, save_dir, factor_range=(0.7, 1.3)):
    image = Image.open(image_path)
    
    enhancer = ImageEnhance.Contrast(image)
    factor = random.uniform(*factor_range)  # Random contrast factor
    contrasted = enhancer.enhance(factor)
    
    # Save
    base_name = os.path.basename(image_path)
    name, ext = os.path.splitext(base_name)
    factor_tag = f"{factor:.2f}".replace('.', '')
    new_name = f"{name}_contrast{factor_tag}{ext}"
    contrasted.save(os.path.join(save_dir, new_name))

def add_random_noise(image_path, save_dir, noise_level=25):
    image = Image.open(image_path).convert('RGB')
    image_array = np.array(image)
    
    # Create noise
    noise = np.random.randint(-noise_level, noise_level + 1, image_array.shape, dtype='int16')
    noisy_array = image_array.astype('int16') + noise
    
    # Clip to valid pixel range
    noisy_array = np.clip(noisy_array, 0, 255).astype('uint8')
    
    noisy_image = Image.fromarray(noisy_array)
    
    # Save
    base_name = os.path.basename(image_path)
    name, ext = os.path.splitext(base_name)
    new_name = f"{name}_noise{ext}"
    noisy_image.save(os.path.join(save_dir, new_name))

def remove_background(image_path, save_dir):
    with open(image_path, "rb") as inp_file:
        input_data = inp_file.read()
        output_data = remove(input_data)
    
    output_image = Image.open(io.BytesIO(output_data)).convert("RGBA")
    
    # Save
    base_name = os.path.basename(image_path)
    name, ext = os.path.splitext(base_name)
    new_name = f"{name}_nobg.png"  # Saving as .png because it supports transparency!
    output_image.save(os.path.join(save_dir, new_name))

def generate_random_background(size, color_range=(100, 255)):
    bg_color = tuple(random.randint(*color_range) for _ in range(3))
    background = Image.new('RGB', size, bg_color)
    return background

# Function to paste PNG onto random background and save multiple versions
def paste_on_multiple_backgrounds(png_path, save_dir, num_variations=5):
    fg_image = Image.open(png_path).convert('RGBA')
    
    base_name = os.path.basename(png_path)
    name, ext = os.path.splitext(base_name)
    
    for i in range(1, num_variations + 1):
        background = generate_random_background(fg_image.size)
        
        combined = Image.alpha_composite(background.convert('RGBA'), fg_image)
        combined = combined.convert('RGB')  # For saving as JPEG (no transparency)

        new_name = f"{name}_bg{i}.jpg"
        combined.save(os.path.join(save_dir, new_name))

In [11]:
for class_folder in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_folder)
    if os.path.isdir(class_path):
        print(f"Augmenting class: {class_folder}")
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            # if (img_file.lower().endswith((".png", ".jpg", ".jpeg")) 
            #     and "_rot" not in img_file 
            #     and "_flip" not in img_file 
            #     and "_scale" not in img_file 
            #     and "_bright" not in img_file 
            #     and "_contrast" not in img_file
            #     and "_noise" not in img_file):
                # rotate_image(img_path, class_path)
                # flip_image(img_path, class_path)
                # scale_image(img_path, class_path)
                # adjust_brightness(img_path, class_path)
                # adjust_contrast(img_path, class_path)
                # add_random_noise(img_path, class_path)
            if "_nobg" in img_file.lower():
                # paste_on_multiple_backgrounds(img_path, class_path)
                flip_image(img_path, class_path)

Augmenting class: 10_Kabitham
Augmenting class: 11_Kadagamugam
Augmenting class: 12_Kangula
Augmenting class: 13_Katharimugam
Augmenting class: 14_Mayuram
Augmenting class: 15_Mirgachirsha
Augmenting class: 16_Mukulam
Augmenting class: 17_Mushti
Augmenting class: 18_Padmakosha
Augmenting class: 19_Pathakam
Augmenting class: 1_Alapadmam
Augmenting class: 20_Sarpashisha
Augmenting class: 21_Shikaram
Augmenting class: 22_Shukathundam
Augmenting class: 23_Simhamugha
Augmenting class: 24_Suchi
Augmenting class: 25_Tamaraichulam
Augmenting class: 26_Thirpathakam
Augmenting class: 27_Trishulam
Augmenting class: 2_Aralam
Augmenting class: 3_Artha chandran
Augmenting class: 4_Arthapathakam
Augmenting class: 5_Brahmaram
Augmenting class: 6_Chandrakala
Augmenting class: 7_Chaturam
Augmenting class: 8_Hamsapaksham
Augmenting class: 9_Hamsasiyam


In [ ]:
# Function to delete certain type of images
def delete_rotated_images(directory):
    for class_folder in os.listdir(directory):
        class_path = os.path.join(directory, class_folder)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                # Check if filename contains the necessary tag 
                if "_scale" in img_file:
                    img_path = os.path.join(class_path, img_file)
                    os.remove(img_path)
                    print(f"Deleted: {img_path}")

In [13]:
delete_rotated_images(train_dir)

Deleted: ../datasets\train\10_Kabitham\14_flip_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_rotated_-10_flip_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_rotated_-10_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_rotated_-15_flip_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_rotated_-15_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_rotated_-5_flip_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_rotated_-5_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_rotated_10_flip_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_rotated_10_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_rotated_15_flip_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_rotated_15_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_rotated_5_flip_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_rotated_5_scale.jpg
Deleted: ../datasets\train\10_Kabitham\14_scale.jpg
Deleted: ../datasets\train\10_Kabitham\15_flip_scale.jpg
Deleted: ../datasets\train\10_Kabitham\15_rota